<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/LearningWithoutForgetting2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

In [ ]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb
import copy
# project classes --------------------------------------------------------------
from IncrementalLearning.cifar100 import ilCIFAR100
from resnet_cifar import resnet32

In [5]:
class lwf(nn.Module):
  def __init__(self,network, random_seed, batch_size):
    super(lwf, self).__init__()
    
    self.batch_size = batch_size
    self.classes_per_task=10
    self.num_tasks=10
    self.LR=2
    self.MOMENTUM=0.9
    self.WEIGHT_DECAY=1e-5
    self.MILESTONES=[49,63]
    self.GAMMA=0.2
    self.numepochs=70
    
    self.original_training_set = ilCIFAR100(self.classes_per_task, random_seed)
    self.original_test_set = ilCIFAR100(self.classes_per_task, random_seed, train=False)
    
    self.model = network.to('cuda')
    self.criterion = nn.BCEWithLogitsLoss()
    self.optimizer = optim.SGD(self.model.parameters(), lr=self.LR, momentum=self.MOMENTUM, weight_decay=self.WEIGHT_DECAY)
    self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=self.MILESTONES, gamma=self.GAMMA)
    
    self.diz = self.original_training_set.get_dict()
    self.task_counter = -1

  def forward(self, x):
    self.model = self.model.cuda()
    return self.model.forward(x)

  def update_parameters(self, train_dataloader):

    self.task_counter += 1 # new incoming task is starting
    old_model = copy.deepcopy(self)
    old_model.eval()
    old_model.to('cuda')

    for epoch in range(self.numepochs):
      for inputs,labels in train_dataloader:
        labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])
        inputs=inputs.to('cuda')
        labels=labels.to('cuda')
        self.optimizer.zero_grad()
        outputs=self.model(inputs)

        new_target=torch.eye(100)[labels] 
        new_target=new_target.to('cuda')

        if self.task_counter == 0:
          loss=self.criterion(outputs,new_target)
        
        else:
          seen_classes = self.task_counter*self.classes_per_task
          #current_classes = seen_classes + self.classes_per_task

          old_target=old_model(inputs)
          old_target=torch.sigmoid(old_target).cuda()
          target = torch.cat((old_target[:,:seen_classes], new_target[:, seen_classes:]), dim=1)
          loss=self.criterion(outputs,target)

        loss.backward()
        self.optimizer.step()
      self.scheduler.step()

        
  # deep copy the model
  def training_lwf(self):
    
    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    current_test_indexes=[]
    
    #diz = self.original_training_set.get_dict()
    acc=[]
    accuracy=0

    for i in range(self.num_tasks):
      train_dataset = Subset(self.original_training_set,train_indices[i])
      current_test_indexes += test_indices[i].tolist()
      test_dataset = Subset(self.original_test_set,current_test_indexes)
      train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
      test_dataloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True)        
      self.train()
      self.update_parameters(train_dataloader)
      self.train(False) # Set Network to evaluation mode
      running_corrects = 0
      y_true = []
      y_pred = []
      for inputs, labels in test_dataloader:
          labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])
          inputs = inputs.to('cuda')
          labels = labels.to('cuda')
          outputs = self.model(inputs)
          _, preds = torch.max(outputs.data, 1)
          running_corrects += torch.sum(preds == labels.data).data.item()
          labels=labels.detach().cpu().numpy()
          labels=labels.tolist()
          y_true.extend(labels)
          preds=preds.detach().cpu().numpy()
          y_pred_tmp = [p for p in preds]
          y_pred.extend(y_pred_tmp)
      accuracy = running_corrects / float(len(test_dataloader.dataset))
      print('Test Accuracy: %.2f' % (100.0 * accuracy))
      print('-' * 80)
      acc.append(accuracy)
    

In [ ]:
lwf(network=resnet32(num_classes=100), batch_size = 128, random_seed = 10).training_lwf()